In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd


# Make table
def process_table(table, groupby, task):
    means = table.groupby(groupby).mean()
    stderrs = table.groupby(groupby).sem()


# Single-signature Gaussians
def curv2sig(curv):
    if curv > 0:
        return "$\\H{2," + str(curv) + "}$"
    elif curv < 0:
        return "$\\S{2," + str(curv) + "}$"
    else:
        return "$\\E{2}$"


class_1man = pd.read_table("../data/results/classification_single_curvature.tsv")
class_1man["task"] = "C"
class_1man["signature"] = class_1man["curvature"].map(curv2sig)
class_1man["dataset"] = "Gaussian"

reg_1man = pd.read_table("../data/results/regression_single_curvature.tsv")
reg_1man["task"] = "R"
reg_1man["signature"] = reg_1man["curvature"].map(curv2sig)
reg_1man["dataset"] = "Gaussian"

# Multiple-signature Gaussians
sig_dict = {
    "H": "$\\H{4}$",
    "E": "$\\E{4}$",
    "S": "$\\S{4}$",
    "HH": "($\\H{2})^2$",
    "HE": "$\\H{2}\\E{2}$",
    "HS": "$\\H{2}\\S{2}$",
    "SE": "$\\S{2}\\E{2}$",
    "SS": "$(\\S{2})^2$",
}

class_sig = pd.read_table("../data/results/classification_signature.tsv")
class_sig["task"] = "C"
class_sig["signature"] = class_sig["signature"].map(sig_dict)
class_sig["dataset"] = "Gaussian"

reg_sig = pd.read_table("../data/results/regression_signature.tsv")
reg_sig["task"] = "R"
reg_sig["signature"] = reg_sig["signature"].map(sig_dict)
reg_sig["dataset"] = "Gaussian"

# Empirical datasets
temp = pd.read_table("../data/results/temperature.tsv")
temp["task"] = "R"
temp["signature"] = "$\\S{2}\\S{1}$"
temp["dataset"] = "Temperature"

fourier = pd.read_table("../data/results/fourier.tsv")
fourier["task"] = "C"
fourier["signature"] = "$(\\S{1})^5$"
fourier["dataset"] = "Neuron 33"

fourier2 = pd.read_table("../data/results/fourier2.tsv")
fourier2["task"] = "C"
fourier2["signature"] = "$(\\S{1})^5$"
fourier2["dataset"] = "Neuron 46"

traffic = pd.read_table("../data/results/traffic_results.tsv")
traffic["task"] = "R"
traffic["signature"] = "$\\E{1}(\\S{1})^4$"
traffic["dataset"] = "Traffic"

# Graph datasets - only keep lowest d_avg
graph_task_dict = {
    "polblogs": "C",
    "citeseer": "C",
    "cora": "C",
    "cs_phds": "R",
}
graph_dataset_names = {
    "polblogs": "PolBlogs",
    "citeseer": "CiteSeer",
    "cora": "Cora",
    "cs_phds": "CS PhDs",
}
graphs = pd.read_table("../data/results/graphs.tsv")
# graphs = graphs.groupby(["embedding", "signature"]).mean().sort_values("d_avg").reset_index().groupby("embedding").first().reset_index()
best_sigs = (
    graphs.groupby(["embedding", "signature"])
    .mean()
    .sort_values("d_avg")
    .reset_index()
    .groupby("embedding")
    .first()
    .reset_index()[["embedding", "signature"]]
)
graphs = pd.merge(graphs, best_sigs, on=["embedding", "signature"])
graphs["signature"] = graphs["signature"].map(sig_dict)
graphs["task"] = graphs["embedding"].map(graph_task_dict)
graphs["dataset"] = graphs["embedding"].map(graph_dataset_names)

# Link prediction datasets
link_dataset_names = {
    "football": "Football",
    "karate_club": "Karate Club",
    "polbooks": "PolBooks",
    "adjnoun": "AdjNoun",
    "dolphins": "Dolphins",
    "lesmis": "Les Mis",
}
links = pd.read_table("../data/results/link_prediction.tsv")
links["task"] = "LP"
# links["signature"] = "$(\\S{2}\\E{2}\\H{2})^2\\E{1}$"
links["signature"] = "$\\S{2}\\E{2}\\H{2}$"
links["dataset"] = links["dataset"].map(link_dataset_names)

# VAE dataset
vae_signature_dict = {
    "blood_cell_scrna": "$\\S{2}\\E{2}(\\H{2})^3$",
    "lymphoma": "$(\\S{2})^2$",
    "cifar_100": "$(\\S{2})^4$",
    "mnist": "$\\S{2}\\E{2}\\H{2}$",
}
vae_dataset_names = {
    "blood_cell_scrna": "Blood",
    "lymphoma": "Lymphoma",
    "cifar_100": "CIFAR-100",
    "mnist": "MNIST",
}
vae = pd.read_table("../data/results/vae.tsv")
vae["task"] = "C"
vae["signature"] = vae["embedding"].map(vae_signature_dict)
vae["dataset"] = vae["embedding"].map(vae_dataset_names)

# Put it all together
class_1man["table"] = "Synthetic (single $K$)"
reg_1man["table"] = "Synthetic (single $K$)"
class_sig["table"] = "Synthetic (multi-$K$)"
reg_sig["table"] = "Synthetic (multi-$K$)"
# temp["table"] = "temperature"
# fourier["table"] = "fourier"
temp["table"] = "Other"
fourier["table"] = "Other"
fourier2["table"] = "Other"
traffic["table"] = "Other"
graphs["table"] = "Graph embeddings"
links["table"] = "Graph embeddings"
vae["table"] = "VAE"

all_data = pd.concat([class_1man, reg_1man, class_sig, reg_sig, temp, fourier, fourier2, traffic, graphs, links, vae])
all_data = all_data.drop(columns=["embedding", "curvature", "d_avg", "seed", "trial"])
all_data

,sklearn_dt,sklearn_rf,product_dt,product_rf,tangent_dt,tangent_rf,knn,ps_perceptron,task,signature,dataset,table
0,0.3450,0.3150,0.3750,0.4050,0.3450,0.3450,0.4250,0.1650,C,"$\S{2,-4.0}$",Gaussian,Synthetic (single $K$)
1,0.3350,0.3500,0.3200,0.3500,0.3700,0.3750,0.3900,0.1450,C,"$\S{2,-4.0}$",Gaussian,Synthetic (single $K$)
2,0.3050,0.3000,0.3850,0.3800,0.3550,0.3900,0.4350,0.1900,C,"$\S{2,-4.0}$",Gaussian,Synthetic (single $K$)
3,0.4150,0.4400,0.5150,0.5200,0.4650,0.5000,0.5400,0.1150,C,"$\S{2,-4.0}$",Gaussian,Synthetic (single $K$)
4,0.3300,0.3450,0.3400,0.4000,0.3300,0.4100,0.3550,0.0700,C,"$\S{2,-4.0}$",Gaussian,Synthetic (single $K$)
...,...,...,...,...,...,...,...,...,...,...,...,...
35,0.2674,0.3038,0.3757,0.4535,0.2594,0.3489,0.4671,0.1105,C,$\S{2}\E{2}\H{2}$,MNIST,VAE
36,0.2773,0.3439,0.2707,0.3870,0.2786,0.3358,0.3648,0.0944,C,$\S{2}\E{2}\H{2}$,MNIST,VAE
37,0.2849,0.3341,0.2991,0.3621,0.2960,0.3378,0.3607,0.1152,C,$\S{2}\E{2}\H{2}$,MNIST,VAE
38,0.2589,0.3246,0.2502,0.3181,0.2594,0.3267,0.3379,0.1394,C,$\S{2}\E{2}\H{2}$,MNIST,VAE


In [11]:
all_data.to_csv("../data/results/all_results.tsv", sep="\t", index=False)

# Generate LaTeX table

In [12]:
all_data = pd.read_table("../data/results/all_results.tsv")

In [16]:
import numpy as np
from scipy.stats import wilcoxon
import pandas as pd

USE_BONFERRONI = True
P_VAL = 0.05
ADJUSTMENT = 10 if USE_BONFERRONI else 1  # 5 choose 2


# Function to calculate mean and standard error
def mean_se(series):
    return series.mean(), series.sem(), series.count()


# Define predictors to compare
group1 = ["product_dt", "sklearn_dt", "tangent_dt", "knn", "ps_perceptron"]
group2 = ["product_rf", "sklearn_rf", "tangent_rf", "knn", "ps_perceptron"]

# Initialize LaTeX symbols for Wilcoxon test results
latex_symbols = {
    "product_dt": "\col{product_dt}{*}",
    "product_rf": "\col{product_dt}{*}",
    "sklearn_dt": "\col{euclidean_dt}{†}",
    "sklearn_rf": "\col{euclidean_dt}{†}",
    "tangent_dt": "\col{tangent_dt}{‡}",
    "tangent_rf": "\col{tangent_dt}{‡}",
    "knn": "\col{knn}{§}",
    "ps_perceptron": "\col{perceptron}{¶}",
}


# Function to format a LaTeX cell with mean ± SE and symbols
def format_latex_cell(mean, se, bold=False, underline=False, symbols=None, regression=False):
    if regression:
        # formatted = f"{mean:.3f}".lstrip("0") + " \scriptsize ± " + f"{se:.3f}".lstrip("0")
        formatted = f"{mean:.3f}".lstrip("0") + " ± " + f"{se:.3f}".lstrip("0")
    else:
        # formatted = f"{mean * 100:.1f}".lstrip("0") + "\scriptsize ± " + f"{se * 100:.1f}".lstrip("0")
        formatted = f"{mean * 100:.1f}".lstrip("0") + " ± " + f"{se * 100:.1f}".lstrip("0")
    if bold:
        formatted = f"\\textbf{{{formatted}}}"
    if underline:
        formatted = f"\\underline{{{formatted}}}"
    if symbols:
        # Remove duplicates
        symbols = sorted(list(set(symbols)))
        formatted += f"\\textsuperscript{{{''.join(symbols)}}}"
    return formatted


# Function to run Wilcoxon test between predictors within group1 and within group2
def run_wilcoxon_tests(group1_vals, group2_vals, regression=False):
    symbols = {key: [] for key in group1 + group2}

    # Compare all pairs within group1
    for i, pred1 in enumerate(group1):
        for pred2 in group1[i + 1 :]:
            if group1_vals[pred1].isnull().all() or group1_vals[pred2].isnull().all():
                continue
            elif (not regression and group1_vals[pred1].mean() > group1_vals[pred2].mean()) or (
                regression and group1_vals[pred1].mean() < group1_vals[pred2].mean()
            ):
                stat, p_value = wilcoxon(group1_vals[pred1], group1_vals[pred2])
                if p_value < P_VAL / ADJUSTMENT:
                    symbols[pred1].append(latex_symbols[pred2])
                    symbols[pred2].append(latex_symbols[pred1])

    # Compare all pairs within group2
    for i, pred1 in enumerate(group2):
        for pred2 in group2[i + 1 :]:
            if group2_vals[pred1].isnull().all() or group2_vals[pred2].isnull().all():
                continue
            # elif group2_vals[pred1].mean() > group2_vals[pred2].mean():
            elif (not regression and group2_vals[pred1].mean() > group2_vals[pred2].mean()) or (
                regression and group2_vals[pred1].mean() < group2_vals[pred2].mean()
            ):
                stat, p_value = wilcoxon(group2_vals[pred1], group2_vals[pred2])
                if p_value < P_VAL / ADJUSTMENT:
                    symbols[pred1].append(latex_symbols[pred2])
                    symbols[pred2].append(latex_symbols[pred1])

    return symbols


# Group by task and dataset
# grouped = all_data.groupby(["table", "task", "dataset", "signature"])
grouped = all_data.groupby(["table", "dataset", "task", "signature"])

# Iterate through each group and process data
results = {}
# for (table, task, dataset, signature), group in grouped:
for (table, dataset, task, signature), group in grouped:
    group_results = {}
    n_samples = {}

    # Calculate mean and SE for each predictor
    for pred in group1 + group2:
        mean, se, n = mean_se(group[pred])
        group_results[pred] = {"mean": mean, "se": se}
        n_samples[pred] = n

    # Run Wilcoxon tests and gather symbols
    symbols = run_wilcoxon_tests(group[group1], group[group2], regression=(task == "R"))

    # Bold the best predictor
    if task == "R":
        best_pred = min(group_results, key=lambda x: group_results[x]["mean"])
        second_best_pred = sorted(group_results, key=lambda x: group_results[x]["mean"])[1]
    else:  # LP and C
        best_pred = max(group_results, key=lambda x: group_results[x]["mean"])
        second_best_pred = sorted(group_results, key=lambda x: group_results[x]["mean"])[-2]

    # Format the results for LaTeX
    # latex_table = {"$n$": max(n_samples.values())}
    latex_table = {}
    for pred in group1 + group2:
        bold = pred == best_pred
        underline = pred == second_best_pred
        latex_table[pred] = format_latex_cell(
            group_results[pred]["mean"],
            group_results[pred]["se"],
            bold=bold,
            underline=underline,
            symbols=symbols.get(pred, []),
            regression=(task == "R"),
        )

    # results[(table, task, dataset, signature)] = latex_table
    results[(table, dataset, task, signature)] = latex_table

# Convert results into a dataframe for display
latex_df = pd.DataFrame.from_dict(results, orient="index")
latex_df.columns = [
    # "$n$",
    "\col{product_dt}{Product DT}",
    "\col{euclidean_dt}{Euclidean DT}",
    "\col{tangent_dt}{Tangent DT}",
    "\col{knn}{$k$-Neighbors}",
    "\col{perceptron}{PS Perceptron}",
    "\col{product_dt}{Product RF}",
    "\col{euclidean_dt}{Euclidean RF}",
    "\col{tangent_dt}{Tangent RF}",
]

# How I want it sorted
sort_order_l1 = [
    "Synthetic (single $K$)",
    "Synthetic (multi-$K$)",
    # "Empirical",
    "Graph embeddings",
    # "VAE embeddings",
    # "Graph",
    "VAE",
    "Other",
]
sort_order_l2 = ["C", "R", "LP"]
sort_order_l3 = [
    "Gaussian mixture",
    "Global temperature",
    "Neural spiking",
    "PolBlogs",
    "CiteSeer",
    "Cora",
    "CS PhDs",
    "Football",
    "Karate Club",
    "PolBooks",
    "AdjNoun",
    "Blood",
    "Lymphoma",
    "CIFAR-100",
    "MNIST",
]
sort_order_l3 = [sig_dict[sig] for sig in sig_dict]

# Make dicts
sort_dict1 = {name: i for i, name in enumerate(sort_order_l1)}
sort_dict2 = {name: i for i, name in enumerate(sort_order_l2)}
sort_dict3 = {name: i for i, name in enumerate(sort_order_l3)}

latex_df = (
    latex_df.assign(
        sort1=latex_df.index.get_level_values(0).map(sort_dict1),
        # sort2=latex_df.index.get_level_values(1).map(sort_dict2),
        sort2=latex_df.index.get_level_values(1).map(sort_dict3),
        # sort3=latex_df.index.get_level_values(2).map(sort_dict3),
        sort3=latex_df.index.get_level_values(2).map(sort_dict2),
    )
    .sort_values(["sort1", "sort2", "sort3"])
    .drop(columns=["sort1", "sort2", "sort3"])
)

# Rotate index labels 1 and 2; add spacing for better alignment
c_dict = {
    "Synthetic (single $K$)": "-5.5cm",
    "Synthetic (multi-$K$)": "-4.5cm",
    "Graph embeddings": "-3cm",
    "VAE": "-1cm",
    "Other": "-1cm",
}
latex_df.index = pd.MultiIndex.from_tuples(
    # [(f"\\rotatebox{{-90}}{{{c}}}", f"\\rotatebox{{-90}}{{{t}}}", d, s) for c, t, d, s in latex_df.index],
    [(f"\\rotatebox{{90}}{{\\hspace{{{c_dict[c]}}}{c}}}", t, d, s) for c, t, d, s in latex_df.index],
    # names=["", "Task", "Dataset", "Signature"],
    names=["", "Dataset", "Task", "Signature"],
)

# Drop columns that never win; rearrange to suit our needs
latex_df = latex_df.drop(
    columns=["\col{tangent_dt}{Tangent DT}", "\col{perceptron}{PS Perceptron}", "\col{tangent_dt}{Tangent RF}"]
)
latex_df = latex_df[
    [
        "\col{knn}{$k$-Neighbors}",
        "\col{euclidean_dt}{Euclidean DT}",
        "\col{euclidean_dt}{Euclidean RF}",
        "\col{product_dt}{Product DT}",
        "\col{product_dt}{Product RF}",
    ]
]

latex_df

/Users/phil/miniforge3/envs/embedders2/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/phil/miniforge3/envs/embedders2/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/Users/phil/miniforge3/envs/embedders2/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/phil/miniforge3/envs/embedders2/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal a

\col{knn}{$k$-Neighbors}  \
                                                   Dataset     Task Signature                                                                  
\rotatebox{90}{\hspace{-5.5cm}Synthetic (single... Gaussian    C    $\E{2}$                28.2 ± .4\textsuperscript{\col{euclidean_dt}{†...   
                                                                    $\H{2,0.25}$           27.8 ± .4\textsuperscript{\col{euclidean_dt}{†...   
                                                                    $\H{2,0.5}$            28.3 ± .5\textsuperscript{\col{perceptron}{¶}\...   
                                                                    $\H{2,1.0}$            28.5 ± .5\textsuperscript{\col{perceptron}{¶}\...   
                                                                    $\H{2,2.0}$            \underline{27.0 ± .5}\textsuperscript{\col{per...   
                                                                    $\H{2,4.0}$            \underline{26.1 ± .5}\textsuperscript{\col{per...   
                                                                    $\S{2,-0.25}$          29.4 ± .5\textsuperscript{\col{perceptron}{¶}\...   
                                                                    $\S{2,-0.5}$           30.6 ± .5\textsuperscript{\col{perceptron}{¶}\...   
                                                                    $\S{2,-1.0}$           \underline{32.6 ± .5}\textsuperscript{\col{per...   
                                                                    $\S{2,-2.0}$           \underline{36.1 ± .5}\textsuperscript{\col{per...   
                                                                    $\S{2,-4.0}$           \textbf{41.6 ± .5}\textsuperscript{\col{percep...   
                                                               R    $\E{2}$                .211 ± .002\textsuperscript{\col{euclidean_dt}...   
                                                                    $\H{2,0.25}$           .207 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\H{2,0.5}$            .206 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\H{2,1.0}$            .214 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\H{2,2.0}$            .211 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\H{2,4.0}$            .215 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\S{2,-0.25}$          .206 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\S{2,-0.5}$           .209 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\S{2,-1.0}$           .207 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\S{2,-2.0}$           .206 ± .003\textsuperscript{\col{euclidean_dt}...   
                                                                    $\S{2,-4.0}$           .198 ± .003\textsuperscript{\col{product_dt}{*...   
\rotatebox{90}{\hspace{-4.5cm}Synthetic (multi-... Gaussian    C    $(\S{2})^2$            \textbf{35.7 ± .5}\textsuperscript{\col{percep...   
                                                                    $\E{4}$                \textbf{34.9 ± .5}\textsuperscript{\col{percep...   
                                                                    $\H{2}\E{2}$           \textbf{40.3 ± .4}\textsuperscript{\col{percep...   
                                                                    $\H{2}\S{2}$           \textbf{38.4 ± .5}\textsuperscript{\col{percep...   
                                                                    $\H{4}$         

In [17]:
latex = latex_df.to_latex(
    # "../data/results/latex_table.tex",
    escape=False,
    # column_format="|>{\centering\\arraybackslash}p{2cm}|>{\centering\\arraybackslash}p{2cm}|p{2cm}|p{2cm}|",
    # column_format=">{\centering\\arraybackslash}p{2cm}>{\centering\\arraybackslash}p{2cm}p{2cm}p{2cm}rrrrrrrrr",
    # column_format=">{\centering\\arraybackslash}p{1cm}>{\centering\\arraybackslash}p{1cm}p{1.5cm}p{1.5cm}llllllll",
    header=True,
)

# Remove all occurrences of "\cline{3-9}"
latex = latex.replace("\\cline{3-9}", "")
latex = latex.replace("\\cline{2-9}", "")

with open("../data/results/latex_table.tex", "w") as f:
    f.write(latex)

In [9]:
# Paste this into the top of the LaTeX file

"""
\toprule
& Dataset & Task & Signature & \col{knn}{\makecell{$k$-Neighbors}} & \col{euclidean_dt}{\makecell{Euclidean \\ DT}} & \col{euclidean_dt}{\makecell{Euclidean \\ RF}} & \col{product_dt}{\makecell{Product \\ DT}} & \col{product_dt}{\makecell{Product \\ RF}} \\
\midrule
"""

"""
\toprule
& Dataset & Task & Signature & \col{knn}{$k$-Neighbors} & \col{euclidean_dt}{Euclidean} & \col{euclidean_dt}{Euclidean} & \col{product_dt}{Product} & \col{product_dt}{Product} \\
& & & & & \col{euclidean_dt}{DT} & \col{euclidean_dt}{RF} & \col{product_dt}{DT} & \col{product_dt}{RF} \\
\midrule
"""

"""
\toprule
& & & & & \col{euclidean_dt}{Euclidean} & \col{euclidean_dt}{Euclidean} & \col{product_dt}{Product} & \col{product_dt}{Product} \\
& Dataset & Task & Signature & \col{knn}{$k$-Neighbors} & \col{euclidean_dt}{DT} & \col{euclidean_dt}{RF} & \col{product_dt}{DT} & \col{product_dt}{RF} \\
\midrule
"""

'\n\toprule\n& Dataset & Task & Signature & \\col{knn}{$k$-Neighbors} & \\col{euclidean_dt}{Euclidean} & \\col{euclidean_dt}{Euclidean} & \\col{product_dt}{Product} & \\col{product_dt}{Product} \\\n& & & & & \\col{euclidean_dt}{DT} & \\col{euclidean_dt}{RF} & \\col{product_dt}{DT} & \\col{product_dt}{RF} \\\n\\midrule\n'